In [1]:
import torch
import os
from torchvision import datasets, transforms, models

In [ ]:
from torchvision import models  
import torch

# Load a pre-trained ResNet model
resnet =  models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
resnet = torch.nn.Sequential(*list(resnet.children())[:-1])  # Remove the last layer
resnet.eval()  # Set to evaluation mode

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet = resnet.to(device)

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 (ResNet input size)
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
])


In [ ]:
train_dir = os.path.join('dataset', 'part_one_dataset', 'train_data')
eval_dir = os.path.join('dataset', 'part_one_dataset', 'eval_data')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
domains = [{} for _ in range(10)]

for j in range(10):
    
    train_path = os.path.join(train_dir, f'{j+1}_train_data.tar.pth')
    t = torch.load(train_path, weights_only = False)
    data = t['data']

    embeds = []
    # Convert to PyTorch tensor
    X_tensor = torch.tensor(data, dtype=torch.float32)  # Convert to tensor
    X_tensor = X_tensor.permute(0, 3, 1, 2)  # Change shape to (2500, 3, 32, 32)

    tensor = X_tensor.float()

    transformed_images = []
    for image in tensor:
        # Convert each image tensor (C, H, W) to PIL Image for transformation
        transformed_image = transform(image)  # Apply the transformations
        transformed_images.append(transformed_image)

    preprocessed_tensor = torch.stack(transformed_images)  # Shape: (2500, 3, 224, 224)

    for i in range(10) : 
        
        preprocessed_batch = preprocessed_tensor[i*250:(i+1)*250]
        preprocessed_batch = preprocessed_batch.to(device)

        # 4. Get the embeddings (feature maps)
        with torch.no_grad():  # Disable gradients for inference
            feature_maps = resnet(preprocessed_batch)  # Shape will be (batch_size, 512, 1, 1)

        # 5. Flatten the feature maps (optional)
        embeddings = feature_maps.view(feature_maps.size(0), -1)  # Flatten to shape (batch_size, embedding_size)

        embeds.append(embeddings)
    
    embeds = torch.vstack(embeds)
    domains[j]['features'] = embeds
    
    torch.save(embeds, f'stuff/train_embeds_{j+1}.pt')

/tmp/ipykernel_85738/4100108068.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  domains[j]['features']  = torch.load(f'stuff/train_embeds_{j+1}.pt', map_location = devi

In [53]:
eval_domains = [{} for _ in range(10)]

for j in range(10):
    
    eval_path = os.path.join(eval_dir, f'{j+1}_eval_data.tar.pth')
    t = torch.load(eval_path, weights_only = False)

    data = t['data'] # both numpy.ndarray
    
    eval_domains[j]['labels'] = t['targets'] if 'targets' in t else None
    
    try:
        eval_domains[j]['features']  = torch.load(f'stuff/eval_embeds_{j+1}.pt', map_location = device)
    except: 
        embeds = []
        # Convert to PyTorch tensor
        X_tensor = torch.tensor(data, dtype=torch.float32)  # Convert to tensor
        X_tensor = X_tensor.permute(0, 3, 1, 2)  # Change shape to (2500, 3, 32, 32)

        tensor = X_tensor.float()

        transformed_images = []
        for image in tensor:
            # Convert each image tensor (C, H, W) to PIL Image for transformation
            transformed_image = transform(image)  # Apply the transformations
            transformed_images.append(transformed_image)

        preprocessed_tensor = torch.stack(transformed_images)  # Shape: (2500, 3, 224, 224)

        for i in range(10) : 
            
            preprocessed_batch = preprocessed_tensor[i*250:(i+1)*250]
            preprocessed_batch = preprocessed_batch.to(device)

            # 4. Get the embeddings (feature maps)
            with torch.no_grad():  # Disable gradients for inference
                feature_maps = resnet(preprocessed_batch)  # Shape will be (batch_size, 512, 1, 1)

            # 5. Flatten the feature maps (optional)
            embeddings = feature_maps.view(feature_maps.size(0), -1)  # Flatten to shape (batch_size, embedding_size)

            embeds.append(embeddings)
        
        embeds = torch.vstack(embeds)
        eval_domains[j]['features'] = embeds
        
        torch.save(embeds, f'stuff/eval_embeds_{j+1}.pt')

/tmp/ipykernel_85738/2390375506.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  eval_domains[j]['features']  = torch.load(f'stuff/eval_embeds_{j+1}.pt', map_location = 